<a href="https://colab.research.google.com/github/nicolenair/capstone/blob/master/anno_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install stanfordnlp
# !pip install import_ipynb


In [0]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import os
import re
# import stanfordnlp
# import import_ipynb
import json

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
os.chdir("/content/gdrive/My Drive/Colab Notebooks/CAPSTONE")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# stanfordnlp.download('en')   # This downloads the English models for the neural pipeline
# nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English

In [0]:
# p = requests.get("https://en.wikipedia.org/wiki/Category:20th-century_British_women_writers")
# soup = BeautifulSoup(p.content, "html.parser")
# authors = soup.findAll("div", { "class" : "mw-category" })[0].text
# auth_list = [i for i in authors.split("\n") if len(i)>1]

# new_url = p + "&pagefrom=auth_list%2C+Priyanga%0APriyanga+Burford"

In [0]:
# file = open("book.id.gables")
# data = json.load(file)
# len(data["characters"][56]["speaking"])

In [0]:
books = ["gables", "avonlea", "island", "dreams", "rilla" ,"ingleside", "poplars", "rainbow"]
urls = ["https://www.gutenberg.org/files/45/45-0.txt", "https://www.gutenberg.org/files/47/47-0.txt", "https://www.gutenberg.org/files/51/51-0.txt", "http://www.gutenberg.org/cache/epub/544/pg544.txt", "http://www.gutenberg.org/cache/epub/3796/pg3796.txt", "http://gutenberg.net.au/ebooks01/0100281.txt", "http://gutenberg.net.au/ebooks01/0100251.txt", "http://www.gutenberg.org/files/5343/5343-0.txt"]

for i in range(len(books)):
    page = requests.get(urls[i])
    file = open("{}.txt".format(books[i]) , "w+")
    file.write(page.text)
    file.close()

file = open("gables.txt", "r")
gables = file.read()
file = open("avonlea.txt", "r")
avonlea = file.read()
file = open("island.txt", "r")
island = file.read()
file = open("dreams.txt", "r")
dreams = file.read()
file = open("rilla.txt", "r")
rilla = file.read()
file = open("ingleside.txt", "r")
ingleside = file.read()
file = open("poplars.txt", "r")
poplars = file.read()
file = open("rainbow.txt", "r")
rainbow = file.read()

In [0]:
#agent

def return_nsubj2(book, character="Anne"): #run corenlp pipeline on each book to annotate quotations
  sentences_with_agent_Anne = []
  sentences_with_agent = []
  if book=="gables":
    doc = nlp(gables)
  elif book=="dreams":
    doc = nlp(dreams)
  elif book=="island":
    doc = nlp(island)
  elif book=="avonlea":
    doc = nlp(avonlea)
  elif book=="rilla":
    doc = nlp(rilla)
  elif book=="ingleside":
    doc = nlp(ingleside)
  elif book=="poplars":
    doc = nlp(poplars)
  elif book=="rainbow":
    doc = nlp(rainbow)
  for i in list(doc.sentences):
    for e in i.dependencies:
  #       print(e[1])
      if e[1]=="nsubj" or e[1]=="agent" :
        if e[2].text==character:
          sentences_with_agent_Anne.append(" ".join([i.dependencies[a][2].text for a in range(len(i.dependencies)) if i.dependencies[a][2].text!=character]))
        else:
          sentences_with_agent.append(" ".join([i.dependencies[a][2].text for a in range(len(i.dependencies)) if i.dependencies[a][2].text!=character]))
        break
  target = []
  target.extend([[1] for i in range(len(sentences_with_agent_Anne))])
  target.extend([[0] for i in range(len(sentences_with_agent))])
  sentences = []
  sentences.extend([[i] for i in sentences_with_agent_Anne])
  sentences.extend([[i] for i in sentences_with_agent])
  return sentences, target


def return_nsubj(book="gables", p = "speaking", tokenid = False):
  file = open("book.id.{}.book".format(book))
  data = json.load(file)
  sentences = []
  target = []
  for i in range(len(data["characters"])):
    if tokenid==True:
      sentences.extend([(s["w"], s["i"]) for s in data["characters"][i][p]])
    else:
      sentences.extend([s["w"] for s in data["characters"][i][p]])
    target.extend([data["characters"][i]["names"] for e in range(len(data["characters"][i][p]))])
      
  return [[i] for i in sentences], [[i] for i in target]

In [0]:
# file = open("book.id.rainbow.book")
# data = json.load(file)
# sentences = []
# target = []
# # print(data["characters"])
# for i in range(len(data["characters"])):
#   #sentences.extend([s["w"] for s in data["characters"][i]["speaking"]])
#   #target.extend([data["characters"][i]["names"] for e in range(len(data["characters"][i]["speaking"]))])
#   print(data["characters"][i]["speaking"])

# full_para_list_rainbow, target_rainbow = [[i] for i in sentences], [[i] for i in target]

In [0]:


for property_ in ["speaking", "agent"]:
  full_para_list_gables, target_gables = return_nsubj("gables", property_, tokenid = True)
  file = open("gables_{}withtoken_list.txt".format(property_), "w")
  for i in range(len(full_para_list_gables)):
    file.write(str(full_para_list_gables[i][0]))
    file.write("\n")
    for e in target_gables[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close() #write to file, with tokens

In [0]:
# # data["characters"][1]["names"][0]
# full_para_list_rainbow

In [0]:
# write to file

for property_ in ["speaking", "agent"]:
  full_para_list_gables, target_gables = return_nsubj("gables", property_)
  full_para_list_dreams, target_dreams = return_nsubj("dreams", property_)
  full_para_list_avonlea, target_avonlea = return_nsubj("avonlea", property_)
  full_para_list_island, target_island = return_nsubj("island", property_)
  full_para_list_rilla, target_rilla = return_nsubj("rilla", property_)
  full_para_list_ingleside, target_ingleside= return_nsubj("ingleside", property_)
  full_para_list_poplars, target_poplars = return_nsubj("poplars", property_)
  full_para_list_rainbow, target_rainbow = return_nsubj("rainbow", property_)
  file = open("gables_{}_list.txt".format(property_), "w")
  for i in range(len(full_para_list_gables)):
    file.write(full_para_list_gables[i][0])
    file.write("\n")
    for e in target_gables[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close()

  #write to file
  file = open("dreams_{}_list.txt".format(property_), "w")
  for i in range(len(full_para_list_dreams)):
    file.write(full_para_list_dreams[i][0])
    file.write("\n")
    for e in target_dreams[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close()

  #write to file
  file = open("avonlea_{}_list.txt".format(property_), "w+")
  for i in range(len(full_para_list_avonlea)):
    file.write(full_para_list_avonlea[i][0])
    file.write("\n")
    for e in target_avonlea[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close()

  file = open("island_{}_list.txt".format(property_), "w+")
  for i in range(len(full_para_list_island)):
    file.write(full_para_list_island[i][0])
    file.write("\n")
    for e in target_island[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close()

  file = open("poplars_{}_list.txt".format(property_), "w+")
  for i in range(len(full_para_list_poplars)):
    file.write(full_para_list_poplars[i][0])
    file.write("\n")
    for e in target_poplars[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close()

  file = open("rilla_{}_list.txt".format(property_), "w+")
  for i in range(len(full_para_list_rilla)):
    file.write(full_para_list_rilla[i][0])
    file.write("\n")
    for e in target_rilla[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close()

  file = open("rainbow_{}_list.txt".format(property_), "w+")
  for i in range(len(full_para_list_rainbow)):
    file.write(full_para_list_rainbow[i][0])
    file.write("\n")
    for e in target_rainbow[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close()

  file = open("ingleside_{}_list.txt".format(property_), "w+")
  for i in range(len(full_para_list_ingleside)):
    file.write(full_para_list_ingleside[i][0])
    file.write("\n")
    for e in target_ingleside[i][0]:
      file.write(str(e["c"])+ "," + str(e["n"])+ ",")
    file.write("\n")
  file.close()


In [0]:
file = open("book.id.gables.book")
data = json.load(file)

In [0]:
for i in data["characters"]:
  for e in i["names"]:
    if e["n"]=="Matthew":
      print (i["speaking"])
      break

[{'w': "`` The little birds sang as if it were The one day of summer in all the year . '' ", 'i': 3723}, {'w': "`` I 'm not expecting a girl , '' ", 'i': 4158}, {'w': "`` Guess there 's some mistake , '' ", 'i': 4205}, {'w': "`` I do n't understand , '' ", 'i': 4275}, {'w': "`` I dare say she 'll be able to explain -- she 's got a tongue of her own , that 's certain . Maybe they were out of boys of the brand you wanted . '' ", 'i': 4314}, {'w': "`` I 'm very glad to see you . I was beginning to be afraid you were n't coming for me and I was imagining all the things that might have happened to prevent you . I had made up my mind that if you did n't come for me to-night I 'd go down the track to that big wild cherry-tree at the bend , and climb up into it to stay all night . I would n't be a bit afraid , and it would be lovely to sleep in a wild cherry-tree all white with bloom in the moonshine , do n't you think ? You could imagine you were dwelling in marble halls , could n't you ? And

In [0]:
for i in data["characters"]:
  for e in i["speaking"]:
    if e["i"]==46229:
      print(e["w"], e["i"])
      break
#   if i["speaking"]!=[]:
#     print(i["speaking"])
#     break

`` acksually never seen anything like it -- it was so white , with awful little red spots in it . ''  46229


In [0]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
sents = sent_tokenize(gables)
len(sents)

5844